In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

In [2]:
'In\tIN\n'.split()

['In', 'IN']

In [3]:
# create vocab from the tarining data and save it in the file (hmm_vocab.txt)
from collections import Counter
with open("WSJ_02-21.pos", 'r') as f:
    lines = f.readlines()
print(lines[:5])
words = [line.split('\t')[0] for line in lines]
freq= Counter(words)
vocab= [key for key, value in freq.items() if (value > 2 and key!='\n')]
vocab.sort()

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']


In [4]:
for i, item in enumerate(freq.items()):
    print(item)
    if i>5:
        break

('In', 1740)
('an', 3143)
('Oct.', 318)
('19', 100)
('review', 58)
('of', 22929)
('``', 6967)


In [5]:
print(set(string.punctuation))

{'$', '<', '>', '?', '.', '_', '*', ',', '#', '!', '/', '(', ':', '\\', '+', '^', '=', '~', ')', '|', '}', '[', '-', '{', '%', ';', "'", '&', '`', '"', '@', ']'}


In [6]:
def assign_unk(word):

    punct = set(string.punctuation)
    
    # Suffixes
    noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffix = ["ate", "ify", "ise", "ize"]
    adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffix = ["ward", "wards", "wise"]

    if any(char.isdigit() for char in word):
        return "--unk_digit--"
    
    elif any(char in punct for char in word):
        return "--unk_punct--"
    
    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    elif any(word.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    elif any(word.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    elif any(word.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    elif any(word.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"
    
    return "--unk--"

In [7]:
print('\n'.split())

[]


In [8]:
def get_word_tag(line, vocab):
    # If line is empty return placeholders for word and tag
    if not line.split():
        word = "--n--"
        tag = "--s--"
    else:
        word, tag = line.split()
#         Handling unknown words 
        if word not in vocab: 
            word = assign_unk(word)
    return word, tag

In [9]:
get_word_tag('\n', vocab)

('--n--', '--s--')

In [10]:
get_word_tag('In\tIN\n', vocab)

('In', 'IN')

In [11]:
get_word_tag('tardigrade\tNN\n', vocab)

('--unk--', 'NN')

In [12]:
def preprocess(vocab, data_fp):
    orig = []
    prep = []

    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):
#             cnt=0, word='The'
#             word= '\n'
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
# 
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
 #             word='The'
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep

In [13]:
# load training data
with open("WSJ_02-21.pos", 'r') as f:
    training_corpus = f.readlines()
# load vocab
with open("vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

In [14]:
voc_l[:10]

['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s"]

In [15]:
# create vocab: dictionary that has the index of the corresponding words
vocab = {} 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       

In [16]:
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break

Vocabulary dictionary, key is the word, value is a unique integer
:0
!:1
#:2
$:3
%:4
&:5


In [17]:
# load in the test corpus
with open("WSJ_24.pos", 'r') as f:
    y = f.readlines()
print(y[0:10])

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']


In [18]:
#corpus without tags, preprocessed
ori, prep = preprocess(vocab, "test.words")     
print(prep[600:800])

['rebound', 'in', 'permits', 'for', 'multifamily', 'units', 'signaled', 'an', 'increase', 'in', 'September', 'starts', ',', 'though', 'activity', 'remains', 'fairly', 'modest', 'by', 'historical', 'standards', '.', '--n--', '--unk_punct--', 'Street', '--n--', 'If', 'the', '--unk_punct--', '--unk_punct--', 'law', "'s", 'fair', ',', 'Why', 'should', 'we', 'not', 'then', 'amend', 'the', '--unk--', 'To', 'require', 'that', 'all', 'employees', 'give', 'Similar', 'notice', 'before', 'they', 'quit', '?', '--n--', '--', '--unk_upper--', 'S.', '--unk_upper--', '.', '--n--', '--unk_upper--', '--unk_upper--', '--n--', 'When', 'research', 'projects', 'are', 'curtailed', 'due', 'to', 'government', 'funding', 'cuts', ',', 'are', 'we', '``', 'caught', 'with', 'our', 'grants', 'down', "''", '?', '--n--', '--', '--unk_punct--', 'Friedman', '.', '--n--', 'Assuming', 'the', 'stock', 'market', 'does', "n't", 'crash', 'again', 'and', 'completely', '--unk--', 'yuppies', 'and', 'trading', 'rooms', ',', 'Amer

In [19]:
print(ori[600:800])

['rebound', 'in', 'permits', 'for', 'multifamily', 'units', 'signaled', 'an', 'increase', 'in', 'September', 'starts', ',', 'though', 'activity', 'remains', 'fairly', 'modest', 'by', 'historical', 'standards', '.', '', 'Two-Way', 'Street', '', 'If', 'the', 'sixty-day', 'plant-closing', 'law', "'s", 'fair', ',', 'Why', 'should', 'we', 'not', 'then', 'amend', 'the', 'writ', 'To', 'require', 'that', 'all', 'employees', 'give', 'Similar', 'notice', 'before', 'they', 'quit', '?', '', '--', 'Rollin', 'S.', 'Trexler', '.', '', 'Candid', 'Comment', '', 'When', 'research', 'projects', 'are', 'curtailed', 'due', 'to', 'government', 'funding', 'cuts', ',', 'are', 'we', '``', 'caught', 'with', 'our', 'grants', 'down', "''", '?', '', '--', 'C.E.', 'Friedman', '.', '', 'Assuming', 'the', 'stock', 'market', 'does', "n't", 'crash', 'again', 'and', 'completely', 'discredit', 'yuppies', 'and', 'trading', 'rooms', ',', 'American', 'television', 'audiences', 'in', 'a', 'few', 'months', 'may', 'be', 'seein

In [20]:
def create_dictionaries(training_corpus, vocab):
    
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    i = 0 
    
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for word_tag in training_corpus:
        
        # Increment the word_tag count
        i += 1
    
        # get the word and tag using the get_word_tag helper function 
        word, tag = get_word_tag(word_tag,vocab) 
        
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
        
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
    
        
    return emission_counts, transition_counts, tag_counts

In [21]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [22]:
print("transition_counts\n")
cnt = 0
for k,v in transition_counts.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break

transition_counts

('--s--', 'IN'):5050
('IN', 'DT'):32364
('DT', 'NNP'):9044
('NNP', 'CD'):1752
('CD', 'NN'):7377
('NN', 'IN'):32885


In [23]:
print("emmision_counts\n")
cnt = 0
for k,v in emission_counts.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break

emmision_counts

('IN', 'In'):1735
('DT', 'an'):3142
('NNP', 'Oct.'):317
('CD', '19'):100
('NN', 'review'):36
('IN', 'of'):22925


In [24]:
print("tag_counts\n")
cnt = 0
for k,v in tag_counts.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break

tag_counts

IN:98554
DT:81842
NNP:91466
CD:36568
NN:132935
``:7092


In [25]:
# get all the POS states
states = sorted(tag_counts.keys())
print(len(states))
print(states)

46
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [26]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'back': print (tup, cnt) 

transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

emission examples: 
(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)

ambiguous word example: 
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


In [27]:
# print the test set actual data
print(y[:10])

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']


In [28]:
# The test set is preprocessed to get only words not tag
print(prep[:10])

['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--']


In [29]:
print(len(y), len(prep))

34199 34199


In [30]:
def predict_pos(prep, y, emission_counts, vocab, states):
    num_correct = 0
    
    # Get the (tag, word) tuples, stored as a set
    all_words = set(emission_counts.keys())
    
    # Get the number of (word, POS) tuples in the corpus 'y'
    total = len(y)
    
# pre--> 'The'
# Get the true label of the word from the test set (y)--> 'The\tDT\n'
# zip--> ('The', 'The\tDT\n')
# word= 'The' and y_tup= 'The\tDT\n'
# y_tup_l= ['The', 'DT']

# Example2- word= '--unk--'
# y='vantage\tNN\n'
#  zip(prep, y)= ('--unk--', 'vantage\tNN\n')
# y_tup_l= ['vantage', 'NN']
# true_label= 'NN'
# word= '--unk--'
    for word, y_tup in zip(prep, y): 
        y_tup_l = y_tup.split()
        if len(y_tup_l) == 2:
            true_label = y_tup_l[1]
        else:
            # If the y_tup didn't contain word and POS, go to next word
            continue
    
       
        
# Example--> test set word = 'The'
# check for all possible pos from states, if that (pos, word) exists in emmision count (from training 
# set)and save the max count in the count_final and get its tag, now compare this tag with the 
# true_label to calculate the accuracy

#Example2--> if '--unk--' in vocab (yes)
# states= ['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
# key1= ('#', --unk--')
# key2= ('$', '--unk--')
# ..
# ..
        count_final = 0
        pos_final = ''
        if word in vocab:
            for pos in states:
                key = (pos,word)
                if key in emission_counts: 
                    count = emission_counts[key]

                    if count>count_final:
                        count_final = count
                        pos_final = pos

            if pos_final == true_label:
                num_correct += 1
    
    accuracy = num_correct / total
    return accuracy

In [31]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


In [32]:
print("tag_counts\n")
cnt = 0
for k,v in tag_counts.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break

tag_counts

IN:98554
DT:81842
NNP:91466
CD:36568
NN:132935
``:7092


In [33]:
all_tags = sorted(tag_counts.keys()) 
print(all_tags)
# print(len(all_tags))
# count_prev_tag = tag_counts[all_tags[i]]
count_prev_tag = tag_counts['#']
print(count_prev_tag)
print(transition_counts[('#', '#')])
print(transition_counts[('#', '$')])

['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
142
0
0


In [34]:
print("transition count dictionary values ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()
trans_keys = set(transition_counts.keys())
cnt = 0

transition count dictionary values 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)



In [35]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
 
    all_tags = sorted(tag_counts.keys()) 
    num_tags = len(all_tags)
    A = np.zeros((num_tags,num_tags))
    
    trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i],all_tags[j])
            if transition_counts: 
                count = transition_counts[key]
            count_prev_tag = tag_counts[all_tags[i]]
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)

    return A

In [36]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
View a subset of transition matrix A
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


In [37]:
print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

emission examples: 
(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)



In [38]:
num_words = len(vocab)
print(num_words)
# print(type(vocab))
# print(vocab)
print("vocab is a dictionary where key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 5:
        break
print(list(vocab)[:5])

23777
vocab is a dictionary where key is the word, value is a unique integer
:0
!:1
#:2
$:3
%:4
&:5
['', '!', '#', '$', '%']


In [39]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
 
    num_tags = len(all_tags)
    all_tags = sorted(tag_counts.keys()) 
    A = np.zeros((num_tags,num_tags))
    
#     trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i],all_tags[j])
            if key in transition_counts.keys(): 
                count = transition_counts[key]
            count_prev_tag = tag_counts[all_tags[i]]
            
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)
    return A

In [40]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    
    num_tags = len(tag_counts)
    all_tags = sorted(tag_counts.keys())
    num_words = len(vocab)
    B = np.zeros((num_tags, num_words))

#     emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key =  (all_tags[i],vocab[j])
            if key in emission_counts.keys():
                count = emission_counts[key]
            count_tag = tag_counts[all_tags[i]]
            
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)
    return B

In [41]:
# creating your emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

# Try viewing emissions for a few words in a sample dataframe
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

# Choose POS tags to show in a sample dataframe
rvals =['CD','NN','NNS', 'VB','RB','RP']

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


In [65]:
print(B)

[[6.03219988e-06 6.03219988e-06 8.56578416e-01 ... 6.03219988e-06
  6.03219988e-06 6.03219988e-06]
 [1.35212298e-07 1.35212298e-07 1.35212298e-07 ... 1.35212298e-07
  1.35212298e-07 1.35212298e-07]
 [1.44034584e-07 1.44034584e-07 1.44034584e-07 ... 1.44034584e-07
  1.44034584e-07 1.44034584e-07]
 ...
 [5.21438963e-06 5.21438963e-06 5.21438963e-06 ... 5.21438963e-06
  5.21438963e-06 5.21438963e-06]
 [4.61514960e-07 4.61514960e-07 4.61514960e-07 ... 4.61514960e-07
  4.61514960e-07 4.61514960e-07]
 [1.40532791e-07 1.40532791e-07 1.40532791e-07 ... 1.40532791e-07
  1.40532791e-07 1.40532791e-07]]


In [42]:
prep[0]

'The'

In [43]:
len(prep)

34199

In [44]:
print(prep[:10])
print(prep[0])
vocab['The']
s_idx = states.index("--s--")
print(states)
print(s_idx)

['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--']
The
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
6


In [45]:
print(math.log(A[6,0])+math.log(B[0,8614]))
print(math.log(A[6,11])+math.log(B[11,8614]))

-22.60982633354825
-4.016237413730867


In [47]:
print(vocab['The'])
print(B[0,8614])
print(B[11,8614])

8614
6.032199882975323e-06
0.0830017285489149


In [48]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
   
    num_tags = len(tag_counts)
    
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    s_idx = states.index("--s--")
    for i in range(num_tags):
        
        if A[s_idx,i] == 0: 
            
            # Initialize best_probs at POS tag 'i', column 0, to negative infinity
            best_probs[i,0] = float('-inf')
        
        # For all other cases when transition from start token to POS tag i is non-zero:
        else:
            
            # Initialize best_probs at POS tag 'i', column 0
            # Check the formula in the instructions above
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]] )
                        

    return best_probs, best_paths

In [66]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [67]:
print(A)

[[7.03997297e-06 7.03997297e-06 7.03997297e-06 ... 7.03997297e-06
  7.03997297e-06 7.03997297e-06]
 [1.35647553e-07 1.35647553e-07 1.35647553e-07 ... 1.35647553e-07
  1.35647553e-07 1.35647553e-07]
 [1.44528595e-07 1.44673124e-04 6.93751711e-03 ... 2.89201719e-04
  3.17977363e-03 2.74618784e-03]
 ...
 [5.95075158e-06 2.38089571e-02 5.95075158e-06 ... 5.95075158e-06
  5.95075158e-06 5.95670233e-03]
 [4.66625541e-07 4.67092167e-04 4.66625541e-07 ... 4.66625541e-07
  4.67092167e-04 1.86696879e-03]
 [1.41003034e-07 1.41144037e-04 1.41003034e-07 ... 1.41003034e-07
  1.12803837e-02 4.23150104e-04]]


In [50]:
# print first column of the best_prob matrix 
for i in range(len(states)):
    print(f"best_probs[{i},0]: {best_probs[i,0]:.4f}") 

best_probs[0,0]: -22.6098
best_probs[1,0]: -23.0766
best_probs[2,0]: -23.5730
best_probs[3,0]: -19.7673
best_probs[4,0]: -24.7433
best_probs[5,0]: -35.2024
best_probs[6,0]: -35.0010
best_probs[7,0]: -34.9920
best_probs[8,0]: -21.3507
best_probs[9,0]: -19.8577
best_probs[10,0]: -21.9210
best_probs[11,0]: -4.0162
best_probs[12,0]: -19.1638
best_probs[13,0]: -21.1062
best_probs[14,0]: -20.4716
best_probs[15,0]: -21.1016
best_probs[16,0]: -21.4958
best_probs[17,0]: -20.4812
best_probs[18,0]: -18.2586
best_probs[19,0]: -23.3972
best_probs[20,0]: -21.9215
best_probs[21,0]: -9.4138
best_probs[22,0]: -21.0305
best_probs[23,0]: -21.0803
best_probs[24,0]: -20.1086
best_probs[25,0]: -33.4819
best_probs[26,0]: -19.4730
best_probs[27,0]: -20.7715
best_probs[28,0]: -20.1173
best_probs[29,0]: -20.5603
best_probs[30,0]: -20.5719
best_probs[31,0]: -32.3037
best_probs[32,0]: -18.0755
best_probs[33,0]: -22.5889
best_probs[34,0]: -19.1586
best_probs[35,0]: -16.0299
best_probs[36,0]: -24.3097
best_probs[37

In [51]:
len(prep)
print(prep[:10])
print(states)
print(states.index('NN'))

['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--']
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
20


In [52]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_forward
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
 
    for i in range(1, len(test_corpus)): 
        for j in range(num_tags):
            
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags):
                prob = best_probs[k,i-1]+math.log(A[k,j]) +math.log(B[j,vocab[test_corpus[i]]])
                
                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
                    
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i

    return best_probs, best_paths

In [53]:
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

In [54]:
prep[1]

'economy'

In [55]:
print(states)

['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [56]:
# Test this function --> check for the word 'economy'
for i in range(len(states)):
    print(f"best_probs[{i},1]: {best_probs[i,1]:.4f}") 

best_probs[0,1]: -24.7822
best_probs[1,1]: -24.5158
best_probs[2,1]: -29.9831
best_probs[3,1]: -25.7122
best_probs[4,1]: -28.7870
best_probs[5,1]: -27.7787
best_probs[6,1]: -30.8835
best_probs[7,1]: -27.9454
best_probs[8,1]: -27.2155
best_probs[9,1]: -28.2268
best_probs[10,1]: -25.2072
best_probs[11,1]: -28.6896
best_probs[12,1]: -33.8392
best_probs[13,1]: -24.7441
best_probs[14,1]: -27.0618
best_probs[15,1]: -23.4693
best_probs[16,1]: -24.1845
best_probs[17,1]: -23.1868
best_probs[18,1]: -33.2349
best_probs[19,1]: -26.2309
best_probs[20,1]: -10.9601
best_probs[21,1]: -24.5507
best_probs[22,1]: -24.1057
best_probs[23,1]: -24.5391
best_probs[24,1]: -28.2113
best_probs[25,1]: -28.2834
best_probs[26,1]: -28.3153
best_probs[27,1]: -27.1986
best_probs[28,1]: -25.8399
best_probs[29,1]: -24.7787
best_probs[30,1]: -22.9700
best_probs[31,1]: -28.3403
best_probs[32,1]: -32.1483
best_probs[33,1]: -29.0336
best_probs[34,1]: -26.3368
best_probs[35,1]: -29.3754
best_probs[36,1]: -27.3496
best_probs[

In [57]:
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_backward
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Get the number of words in the corpus
    # which is also the number of columns in best_probs, best_paths
    m = best_paths.shape[1] 
    
    # Initialize array z, same length as the corpus
    z = [None] * m
    
    # Get the number of unique POS tags
    num_tags = best_probs.shape[0]
    
    # Initialize the best probability for the last word
    best_prob_for_last_word = float('-inf')
    
    # Initialize pred array, same length as corpus
    pred = [None] * m
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    ## Step 1 ##
    
    # Go through each POS tag for the last word (last column of best_probs)
    # in order to find the row (POS tag integer ID) 
    # with highest probability for the last word
    for k in range(num_tags): # complete this line

        # If the probability of POS tag at row k 
        # is better than the previosly best probability for the last word:
        if best_probs[k,-1]>best_prob_for_last_word: # complete this line
            
            # Store the new best probability for the lsat word
            best_prob_for_last_word = best_probs[k,-1]
    
            # Store the unique integer ID of the POS tag
            # which is also the row number in best_probs
            z[m - 1] = k
            
    # Convert the last word's predicted POS tag
    # from its unique integer ID into the string representation
    # using the 'states' dictionary
    # store this in the 'pred' array for the last word
    pred[m - 1] = states[k]
    
    ## Step 2 ##
    # Find the best POS tags by walking backward through the best_paths
    # From the last word in the corpus to the 0th word in the corpus
    for i in range(len(corpus)-1, -1, -1): # complete this line
        
        # Retrieve the unique integer ID of
        # the POS tag for the word at position 'i' in the corpus
        pos_tag_for_word_i = best_paths[np.argmax(best_probs[:,i]),i]
        
        # In best_paths, go to the row representing the POS tag of word i
        # and the column representing the word's position in the corpus
        # to retrieve the predicted POS for the word at position i-1 in the corpus
        z[i - 1] = best_paths[pos_tag_for_word_i,i]
        
        # Get the previous word's POS tag in string form
        # Use the 'states' dictionary, 
        # where the key is the unique integer ID of the POS tag,
        # and the value is the string representation of that POS tag
        pred[i - 1] = states[pos_tag_for_word_i]
        
     ### END CODE HERE ###
    return pred

In [58]:
# Run and test your function
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


In [59]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The third word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN



In [60]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(pred, y):
    '''
    Input: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Output: 
        
    '''
    num_correct = 0
    total = 0
    
    # Zip together the prediction and the labels
    for prediction, y in zip(pred, y):
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # Split the label into the word and the POS tag
        word_tag_tuple = y.split()
        
        # Check that there is actually a word and a tag
        # no more and no less than 2 items
        if len(word_tag_tuple)!=2: # complete this line
            continue 

        # store the word and tag separately
        word, tag = word_tag_tuple
        
        # Check if the POS tag label matches the prediction
        if prediction == tag: # complete this line
            
            # count the number of times that the prediction
            # and label match
            num_correct += 1
            
        # keep track of the total number of examples (that have valid labels)
        total += 1
        
        ### END CODE HERE ###
    return num_correct/total

In [61]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9528
